## Урок 2. Парсинг HTML. BeautifulSoup, MongoDB

1) Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта `superjob.ru` и `hh.ru`. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    *Наименование вакансии
    *Предлагаемую зарплату (отдельно мин. и отдельно макс. и отдельно валюта)
    *Ссылку на саму вакансию        
    *Сайт откуда собрана вакансия
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью `dataFrame` через `pandas`.

### Решение
В домашней работе выполнял поиск вакансии для ***C++ Developer***. 
Каждый из сайтов в названии вакансии заменяет часть символов из строки поискового запроса специфическими символоами форматирования. Поэтому, чтобы не увеличиваить объем кода, я не стал делать преобразования, а просто подставил эти строки как есть.

In [1]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import pandas as pd
import json

In [2]:
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36',
    'charset': 'utf-8'
}

In [3]:
vacancy = [] # Словарь в котором будет храниться список вакансий.  

In [4]:
def selary_analizer(selary_txt):  # возвращает массив содержащий значение зарбплаты.
    min_selary = None
    max_selary = None
    money_type = None
    symbols = ['от', 'до']
    # print(selary_txt)
    if selary_txt != '' and selary_txt != 'По договорённости':  # 'проверяем наличие ключевых символов в строке'
        selary_txt = selary_txt.replace('—\xa0', '')
        selary_txt = selary_txt.replace('-', ' ')
        selary_txt = selary_txt.replace('\xa0', ' ')
        selary_txt = selary_txt.replace('/месяц', '')
        selary_array = selary_txt.split(' ')
        money_type = selary_array[-1].rstrip('.')

        if selary_array[0] == symbols[0]: # от
            min_selary = selary_array[1] + selary_array[2]
        elif (selary_array[0] == symbols[1]):  # до
            max_selary = selary_array[1] + selary_array[2]
        elif (len(symbols) < 4):  # до
            max_selary = selary_array[0] + selary_array[1]
        else:                                           
            min_selary = selary_array[0] + selary_array[1]
            max_selary = selary_array[2] + selary_array[3]

    return [min_selary, max_selary, money_type]

### Поиск Вакансии на сайте  https://hh.ru

In [5]:
main_link = 'https://izhevsk.hh.ru/search/vacancy?L_is_autosearch=false&area=113&clusters=true&enable_snippets=true'
page = 0
is_next_page = True  # Флаг поназываюций что есть еще страницы.
# Выполняем переход на следующую страницу.
# Может существовать как минимум одна страница с запросом.

In [6]:
text = 'C%2B%2B+Developer'  # 'Machine+Learning+Engineer'

In [7]:
i = 0
while is_next_page == True:
    response = requests.get(f'{main_link}&text={text}&page={page}', headers=header).text
    soup = bs(response, 'lxml')

    btn_next = soup.find('a', {'class': "HH-Pager-Controls-Next"})

    if btn_next != None:
#         print(f'Cтраница {page + 1}',end=" ")
        page += 1
    else:
#         print(f'Cтраница {page + 1}',end=" ")
        is_next_page = False

    vacancy_block = soup.find('div', {'class': 'vacancy-serp'})
    divs = vacancy_block.findChildren(recursive=False)


    for vk in divs:
        vacancy_data = {}
        i += 1
        try:
            if vk.attrs['class'][0] != 'serp-special':  # Проверка на то не являетс яли даггый тег рекламой.
                vacancy_name = vk.find('div', {'class': 'vacancy-serp-item__info'}).getText()
                # Ссылка на вакансию.
                vk_href = vk.find('a', {'class': 'bloko-link HH-LinkModifier'})['href']
                # Нужно создать функцию для разбиения строки. на составляющие.
                selary_txt = vk.find('div', {'class': 'vacancy-serp-item__sidebar'}).getText()
                # Название фирмы.
                vk_employer = vk.find('a', {'data-qa': "vacancy-serp__vacancy-employer"}).getText()
                vk_employer_id = 'https://izhevsk.hh.ru' + vk.find('a', {'data-qa': "vacancy-serp__vacancy-employer"})[
                    'href']
                # Город расположения.
                vk_location = vk.find('span', {'data-qa': "vacancy-serp__vacancy-address"}).getText()
                # Дата публикации.
                vacancy_date = vk.find('span', {'data-qa': "vacancy-serp__vacancy-date"}).getText()

                min_selary, max_selary, money_type = selary_analizer(selary_txt)

                vacancy_data['name'] = vacancy_name
                vacancy_data['employer'] = vk_employer
                vacancy_data['location'] = vk_location
                vacancy_data['min_selary'] = min_selary
                vacancy_data['max_selary'] = max_selary
                vacancy_data['money_type'] = money_type
                # vacancy_data['publicatin_date'] = vacancy_date
                vacancy_data['href'] = vk_href
                vacancy_data['employer_id'] = vk_employer_id
                vacancy_data['source'] = 'https://hh.ru/'
                vacancy.append(vacancy_data)
        except Exception as ex:
            print("")
            print('Ошибка при анализе вакансии ', ex)
        print("", end="\r")
        print(f'Найдено вакансий: {i}',end=" ")

Найдено вакансий: 2137   
Ошибка при анализе вакансии  'NoneType' object has no attribute 'getText'
Найдено вакансий: 2200  

### Поиск вакансии на сайте https://russia.superjob.ru

In [8]:
main_link = 'https://russia.superjob.ru/vacancy/search/'
page = 1 # Отсчет начинается с 1. 
is_next_page = True  # Флаг поназываюций что есть еще страницы.

In [9]:
text = 'C%2B%2B%20Developer' #'Machine%20Learning%20Engineer'

In [10]:
i = 0
while (is_next_page == True):
    response = requests.get(f'{main_link}?keywords={text}&page={page}', headers=header).text

    soup = bs(response, 'lxml')

    btn_next = soup.findAll('span', {'class': "qTHqo _1mEoj _2h9me DYJ1Y _2FQ5q _2GT-y"})
    if 'Дальше' == btn_next[-1].getText():
#         print(f'Cтраница {page}')
        page += 1
    else:
#         print(f'Cтраница {page}')
        is_next_page = False

    vacancy_hrefs = soup.find('script',
                              {'type': "application/ld+json"}).string  # Преобразуем полученное значение в  json.

    v_links = json.loads(vacancy_hrefs)
    v_link_list = []
    for v_link in v_links['itemListElement']:
        v_link_list.append(v_link['url'])


    # Последовательно загружаем сайты с описанием вакансии и считываем с него информацию.

    for v_link in v_link_list:
        i += 1
        vacancy_data = {}
        try:
            v_response = requests.get(v_link, headers=header).text

            v_soup = bs(v_response, 'lxml')
            vs = v_soup.find('div', {'class': '_3Qutk'})
            vsk = vs.find_next('div', {'class': '_3MVeX'})

            vacancy_name = vsk.contents[1].getText()  
            # Ссылка на вакансию.
            vk_href = v_link
            selary_txt = vsk.contents[4].getText()
            # Город расположения.
            vk_location = vsk.contents[2].getText()

            vs2 = vs.find_next('div', {'class': '_3zucV undefined'})
            # Название фирмы.
            vk_employer_id = None
            vk_employer = vs2.find('a', {'class': 'icMQ_'})
            if vk_employer is not None:
                vk_employer_id = 'https://russia.superjob.ru' + vk_employer['href']
                vk_employer = vk_employer.getText()
            else:
                vk_employer = vs.find('span', {'class': '_3mfro _1hP6a _2JVkc _2VHxz'}).getText()

            min_selary, max_selary, money_type = selary_analizer(selary_txt)
            vacancy_data['name'] = vacancy_name
            vacancy_data['employer'] = vk_employer
            vacancy_data['location'] = vk_location
            vacancy_data['min_selary'] = min_selary
            vacancy_data['max_selary'] = max_selary
            vacancy_data['money_type'] = money_type
            # vacancy_data['publicatin_date'] = vacancy_date # не смог найти где дата публикации указана.
            vacancy_data['href'] = vk_href
            vacancy_data['employer_id'] = vk_employer_id
            vacancy_data['source'] = 'https://russia.superjob.ru'
            vacancy.append(vacancy_data)
        except Exception as ex:
            print("")
            print('Ошибка при анализе вакансии ', ex)
        print("", end="\r")
        print(f'Найдено вакансий: {i}',end=" ")

Найдено вакансий: 30 

In [11]:
df = pd.DataFrame(vacancy)

In [12]:
df.head()

,name,employer,location,min_selary,max_selary,money_type,href,employer_id,source
0,Программист С++/Драйверист,Searchinform,Ростов-на-Дону,None,100000,руб,https://izhevsk.hh.ru/vacancy/36712554?query=C...,https://izhevsk.hh.ru/employer/534283,https://hh.ru/
1,Программист-разработчик,Филиал компании Ол Фюнерал Сервисиз Москва,Петрозаводск,None,60000,руб,https://izhevsk.hh.ru/vacancy/37614689?query=C...,https://izhevsk.hh.ru/employer/4702861,https://hh.ru/
2,"Senior Software Engineer, Database Engineering...",Cube.js,Москва,None,None,None,https://izhevsk.hh.ru/vacancy/36976348?query=C...,https://izhevsk.hh.ru/employer/4573853,https://hh.ru/
3,Senior Node.js Engineer (Cube.js Core),Cube.js,Москва,None,None,None,https://izhevsk.hh.ru/vacancy/37279695?query=C...,https://izhevsk.hh.ru/employer/4573853,https://hh.ru/
4,"C++ developer (deep learning, CV products)",Tech5,Москва,None,150000,руб,https://izhevsk.hh.ru/vacancy/37508809?query=C...,https://izhevsk.hh.ru/employer/3828175,https://hh.ru/


In [13]:
df.describe()

,name,employer,location,min_selary,max_selary,money_type,href,employer_id,source
count,2029,2029,2029,296,434,730,2029,2029,2029
unique,1433,918,329,52,63,3,2029,919,2
top,Инженер-программист,HUAWEI,Москва,100000,80000,руб,https://izhevsk.hh.ru/vacancy/37146096?query=C...,https://izhevsk.hh.ru/employer/2873,https://hh.ru/
freq,59,40,408,36,41,699,1,40,1999


##### Сохраняем результаты на в файл.

In [14]:
df.to_csv('Vacanсies.csv', index=False)